In [4]:
# Importing all required methods
%matplotlib inline

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.optim import Adam
import os
import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.preprocessing import MultiLabelBinarizer
import myutils

In [5]:
# Directory where mp3 are stored.
AUDIO_DIR = os.environ.get('AUDIO_DIR')
current_directory = os.getcwd()

# Print the current working directory
print("Current directory:", current_directory)

# Load metadata and features.
tracks = myutils.load('data/fma_metadata/tracks.csv')
genres = myutils.load('data/fma_metadata/genres.csv')
features = myutils.load('data/fma_metadata/features.csv')
echonest = myutils.load('data/fma_metadata/echonest.csv')

Current directory: c:\Users\Tobey\Desktop\csc413-final-project


In [87]:
# get tracks for small data set
small_tracks = tracks[tracks['set', 'subset'] <= 'small']

# get features for small dataset 
small_features = features[tracks['set', 'subset'] <= 'small']

# select top genre column
hottest_genres = small_tracks["track"]["genre_top"]
hottest_genres = hottest_genres.to_frame()
hottest_genres = hottest_genres.dropna()

# get indices
ids = hottest_genres.index.values.reshape(-1, 1)
# note: track ids for tracks that ONLY have a valid top_genre

# Extract categories from categorical column
categories = hottest_genres['genre_top'].cat.categories

# Convert categories to a dictionary mapping category values to integer indices
categories_dict = {category: i for i, category in enumerate(categories)}

# convert to integer codes
hottest_genres['genre_top'] = hottest_genres['genre_top'].cat.codes

# convert to array
hottest_genres = np.array(hottest_genres)

# create labels data 
labels = np.zeros((hottest_genres.shape[0], 16))
labels[np.arange(hottest_genres.shape[0]), hottest_genres.flatten()] = 1

In [ ]:
# TODO: filter the feature dataframe to only have the tracks in 'ids':

In [ ]:
# TODO: Create 8000x512 for track <-> feature data storage.
# features.head(5).style.format('{:.2f}')

In [ ]:
# TODO: Create 8000xspectrogram_data for each track